This notebook unnest the mapbox api json responses into dataframes in order to join the route responses with their corresponding source/target stations pairs

In [99]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import simplejson

In [98]:
#import chunked api responses
data = json.load(open('routes30k.json'))

In [100]:
#replacement values for api responses that came with errors
replace = {'code': 'NoValue',
  'routes': [{'distance': 'NoValue',
    'duration': 'NoValue',
    'geometry': {'coordinates': [['NoValue', 'NoValue']],
     'type': 'NoValue'},
    'legs': [{'distance': 'NoValue',
      'duration': 'NoValue',
      'steps': [],
      'summary': '',
      'weight': 'NoValue'}],
    'weight': 'NoValue',
    'weight_name': 'NoValue'}],
  'uuid': 'NoValue',
  'waypoints': [{'location': ['NoValue', 'NoValue'],
    'name': 'Central Park West'},
   {'location': ['NoValue', 'NoValue'], 'name': 'NoValue'}]}

for number in data:
    for key in data[number]:
        if 'message' in key:
            data[number] = replace


In [101]:
#create a new json file with the updated responses
json.dump(data,open('request_new.json','w'),indent=2)
data = json.load(open('request_new.json'))

In [53]:
#unnest api responses
status = []
routes = []
for trip in data:
    status.append(data[trip]['code'])
    routes.append(data[trip]['routes'])

In [102]:
#contnue to unnest route api responses into the next level
#transform these responses into individual dataframes
dist = []
dur = []
geom = []

i = 0

while i < 2413:
    dist.append(routes[i][0]['distance'])
    dur.append(routes[i][0]['duration'])
    geom.append(routes[i][0]['geometry'])
    i=i+1

distance = pd.DataFrame(dist)
duration = pd.DataFrame(dur)
geometry = pd.DataFrame(geom)

In [56]:
#rename route responses columns
distance = distance.rename(columns={distance.columns[0]:'distance'})
duration = duration.rename(columns={duration.columns[0]:'duration'})
geometry = geometry.rename(columns={geometry.columns[0]:'geometry'})

In [57]:
#rename status response oclumn
status = pd.DataFrame(status)
status = status.rename(columns={status.columns[0]:'status'})

In [58]:
#join all dataframes into one
mapbox_dirs = status.join(distance)
mapbox_dirs = mapbox_dirs.join(duration)
mapbox_dirs = mapbox_dirs.join(geometry)

In [59]:
#save dataframe into a csv
mapbox_dirs.to_csv('map_30k.csv')

In [63]:
#import all saved csvs
map10 = pd.read_csv('map_10k.csv')
map20 = pd.read_csv('map_20k.csv')
map30 = pd.read_csv('map_30k.csv')
map40 = pd.read_csv('map_40k.csv')
map50 = pd.read_csv('map_50k.csv')
map60 = pd.read_csv('map_60k.csv')
map70 = pd.read_csv('map_70k.csv')

In [65]:
#append all responses dataframes to eachother
routes_info = pd.concat([map10,map20,map30,map40, map50, map60, map70])

In [66]:
routes_info = routes_info.reset_index()

In [67]:
routes_info = routes_info.drop(columns=['index','Unnamed: 0'])

In [68]:
#import dataframe of the 70k stations sent to the mapbox api
api_stations = pd.read_csv('stations_sent_to_api.csv')

In [69]:
#join station directions with the routes dataframe
station_directions = routes_info.join(api_stations)

In [72]:
#filter out "no value" responses
station_directions = station_directions[station_directions['duration']!='NoValue']
station_directions.to_csv('directions.csv')